In [25]:
import pandas as pd
import numpy as np
import os

INPUT_FILE = r"C:\Users\omkum\Downloads\output_file.csv"
OUTPUT_DIR = r"C:\Users\omkum\Downloads\assiresult"

MONTHS_REQUIRED = 24

SMA_WINDOWS = [10, 20]
EMA_WINDOWS = [10, 20]

os.makedirs(OUTPUT_DIR, exist_ok=True)


def load_data(file_path):
    df = pd.read_csv(file_path)
    df['date'] = pd.to_datetime(df['date'])
    df.sort_values(['ticker', 'date'], inplace=True)
    return df


# Monthly OHLC Aggregation

def resample_monthly(df):
    monthly = (
        df.set_index('date')
          .groupby('ticker')
          .resample('M')  
          .agg({
              'open': 'first',
              'high': 'max',
              'low': 'min',
              'close': 'last',
              'adjclose': 'last',   # monthly adjusted close
              'volume': 'sum'
          })
          .reset_index()
    )
    return monthly


# Indicator Calculations

def calculate_sma(series, window):

    return series.rolling(window=window).sum() / window

def calculate_ema(series, window):
 
    multiplier = 2 / (window + 1)
    ema_values = [np.nan] * len(series)

    # using SMA as the "Previous Day's EMA"
    first_sma = series.iloc[:window].sum() / window
    ema_values[window - 1] = first_sma

    for i in range(window, len(series)):
        ema_values[i] = (
            (series.iloc[i] - ema_values[i - 1]) * multiplier
            + ema_values[i - 1]
        )

    return pd.Series(ema_values, index=series.index)


# Add Indicators Per Ticker

def add_indicators(df):
    close_prices = df['close']

    for w in SMA_WINDOWS:
        df[f'SMA_{w}'] = calculate_sma(close_prices, w)

    for w in EMA_WINDOWS:
        df[f'EMA_{w}'] = calculate_ema(close_prices, w)

    return df


# Write Output Files

def write_ticker_files(df):
    output_columns = [
        'date', 'ticker',
        'open', 'high', 'low', 'close', 'adjclose', 'volume',
        'SMA_10', 'SMA_20',
        'EMA_10', 'EMA_20'
    ]

    for ticker, data in df.groupby('ticker'):
        if len(data) < MONTHS_REQUIRED:
            raise ValueError(f"{ticker} has less than {MONTHS_REQUIRED} months of data")

        data = data.tail(MONTHS_REQUIRED)
        output_path = os.path.join(OUTPUT_DIR, f"result_{ticker}.csv")
        data[output_columns].to_csv(output_path, index=False)
        

def run_pipeline():
    # Load the data
    df = load_data(INPUT_FILE)

    # Resample daily data to monthly OHLC
    monthly_df = resample_monthly(df)

    # Add indicators per ticker
    monthly_df = (
        monthly_df
        .groupby('ticker', group_keys=False)
        .apply(add_indicators)
    )

    # Write output files
    write_ticker_files(monthly_df)

    print("Processing completed successfully.")

# print(df)
# df['adjclose']
run_pipeline()

C:\Users\omkum\AppData\Local\Temp\ipykernel_3004\3986076021.py:29: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample('M')  # standard month-end


Processing completed successfully.


C:\Users\omkum\AppData\Local\Temp\ipykernel_3004\3986076021.py:110: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(add_indicators)
